# Hyperparameter Tuning

### Step 1: What can be changed?
if best optimizer is not adamax, remember to change it in nn_cl_bo and also change the hyperparameter to be tuned
* hyperparameter values
* Number of iteration before actually searching
* Number of searching iteration
#### What can be change (not yet done)
* Activation function tuning (now is tf.nn.relu)
* number of hidden layer (can use iteration/loop)

In [1]:
params_nn ={
     'N_hidden_nodes': (10, 2000),
     'l_rate':(0.0001, 0.01),
     'Batch_size':(1, 64),
     'epochs':(50, 1000),
 }
no_iter_before_conclude=5
no_iter_conclude=3
datapathway=r"C:\Users\wingt\Downloads\S11_along_BD_205.csv"
xstart=3            #column of xstart
xstop=5            #column of xend
ystart=6           #column of ystart
yname='Stress'      # name of the ultimate output array
yparameter='Position'       #position here but it will be what is the parameter to be tuned
xcolumn=['Travel_length', 'Welding_speed', 'Net_energy_input'] #name of inputs

### Step 2: Import library

In [2]:
import numpy as np
import pandas as pd
import warnings
import os
import random
import pickle
# import ML related libraries
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import SGD, Adam
from keras.models import Model
from keras.layers import Dense, Input, Dropout
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from bayes_opt import BayesianOptimization
# import plotting related libraries
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from scipy import interpolate

### Step 3: import data + preprocessing
import the data using pd.read_csv
only get columns required
assign column names
get y data for training as well

In [3]:
def data_import(csv_file_name):
    '''
    used to import dataset and split training and test dataset
    csv_file_name is the dateset root
    reture traing and test datasets
    '''
    raw_data = pd.read_csv(csv_file_name,header=None).dropna()
    x = raw_data.iloc[:,xstart:(xstop+1)]
    #x.columns = xcolumn
    y = raw_data.iloc[:,ystart:]
    return x, y

### Step 4: Neural Network training for Bayesian Optimisation
use a function for defning the machine name and parameters for the machine
use KerasRegressor to build the machine with specified hyperparameters
use processed training input and output (will define function to make them later) for training the model
history is the name of the model
return a scoe showing cross validation mse

In [4]:
def nn_cl_bo(N_hidden_nodes, l_rate, Batch_size, epochs):
    '''
    Using certain number of neurons, learning rate and Batch size
    to train a NN and return its cross validation MSE
    inputs:
        N_hidden_nodes: number of neurons
        l_rate: learning rate
        Batch_size: Batch size
    output:
        score: cross validation MSE
    '''
    N_hidden_nodes = round(N_hidden_nodes)
    Batch_size = round(Batch_size)
    def nn_cl_fun():
        nn = keras.Sequential([
            keras.layers.Dense(N_hidden_nodes, activation=tf.nn.relu, input_shape=(4,)),
        # keras.layers.Dropout(0.5),
            keras.layers.Dense(1, activation='linear')
        ])
        nn.compile(loss='mse',
                    optimizer=tf.keras.optimizers.Adamax(learning_rate=l_rate, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.0),
                    metrics=['mse'])
        return nn
    nn = KerasRegressor(build_fn=nn_cl_fun, epochs=epochs, batch_size=Batch_size,
                        verbose=0)
    history = nn.fit(H_X_train, H_Y_train, batch_size=Batch_size, epochs=1000, 
                        verbose=0, validation_split=0.1)
    score = -history.history['val_loss'][-1]
    return score

In [5]:
def create_model_2(N_hidden_nodes, l_rate, Batch_size, epochs):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    model = keras.Sequential([
        keras.layers.Dense(N_hidden_nodes, activation=tf.nn.leaky_relu, input_shape=(4,)),
        keras.layers.Dense(21,activation='linear')
    ])
    optimizer=tf.keras.optimizers.Adamax(learning_rate=l_rate, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.0)
    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mse'])
    model.summary()
    model.fit(Proc_X_train, Proc_Y_train, batch_size=Batch_size, epochs=epochs, 
                        verbose=0, validation_split=0.1)
    return model


### Step 5: Uniform Spaced Sampling (Just for this database)

In [6]:
def uniformly_spaced_sampling(y_label, y_label_new, y):
    '''
    fit and interpolate
    '''
    f = interpolate.interp1d(y_label, y, kind='linear') # linear interpolation function use first row of y and all of y -> linear interpolate location and y value
    ynew=pd.DataFrame(f(y_label_new)) #first row of y (linear distributed) -> linearly interpolate(predict) when in evenly distributed location, what r y
    return ynew

### Step 6: assign column names to data set 
can see only one column of y here

In [7]:
def data_melt(x, y):
    '''
    Add position as input
    input:
        x: old input 
        y: old output
    output:
        x_new: new input 
        y_new: new output
    '''
    y_label_str = [str(x) for x in y_label_new] # make a list of string of y_label_new
    dataset = pd.concat([x, y],axis=1, ignore_index=True)
    col_names = xcolumn + y_label_str
    dataset.columns = col_names
    dataset = dataset.melt(id_vars=xcolumn, 
        var_name=yparameter, 
        value_name=yname) #for given welding parameter, in a given location, the stress is in this dataset
    x_new = dataset.iloc[:, 0:(len(xcolumn)+1)]
    y_new = dataset.iloc[:, (len(xcolumn)+1)]
    return x_new, y_new

### Step 7: Define a function getting y(output) label (not for us again)
use y in the csv
the first row of that csv is y_lab
y_lab_new is min to max of y_lab in length of y_lab

In [8]:
def get_y_lable(root):
    '''
    Create an array for uniform interval depth 
    (Used biased mesh in simulaiton, so the sample point depth is not uniform) 
    input:
        root: file root contains the sample point depth information
    outputs:
        y_lab: old sample point position array
        y_lab_new: new sample point position array
    '''
    x, y = data_import(root)
    y_lab = y.iloc[0, :]
    y_lab_new = np.linspace(round(min(y_lab),2), 
                              round(max(y_lab),2), 
                              round(len(y_lab),2))
    return y_lab, y_lab_new
    

### Step 8: Hyperparameter tuning: Data Prepocessing function

In [9]:
def H_pre_processing(model_type, x, y):
    '''
    Data preprocessing (Uniformly spaced sampling, normalisation, train test split)
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)        
        x: a dataframe of inputs of the whole dataset
        y: a dataframe of outputs of the whole dataset
    outputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        Y_test: Test output
    '''
      # uniformly_spaced_sampling
    global y_label, y_label_new
    y_label, y_label_new = get_y_lable(r'C:\Users\wingt\Downloads\benchmark_BD1.csv')
    y = uniformly_spaced_sampling(y_label, y_label_new, y)

    # Split dataset
    # X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=1/125.0, random_state=3)
    Y_train = uniformly_spaced_sampling(y_label, y_label_new, y)
    # data reconstruction
    if 'ANN2' in model_type:
        X_train, Y_train = data_melt(x, y)

    # Normalization
    global scaler_X, scaler_Y
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    scaled_train_X = scaler_X.fit_transform(X_train.to_numpy())
    if Y_train.ndim == 1:
      Y_train = np.array(Y_train).reshape(-1,1)#column array
    scaled_train_Y = scaler_Y.fit_transform(Y_train)
    H_X_train = scaled_train_X
    H_Y_train = scaled_train_Y
    return model_type, H_X_train, H_Y_train

### Step 9: fix random seed

In [10]:
def seed_tensorflow(seed):
    '''
    Fix ramdom seed
    '''
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['PYTHONHASHSEED'] = str(seed)
warnings.filterwarnings('ignore')
seed_tensorflow(42)

### Step 10: Import data from the address and tune hyperparameter
* print out the processes
* print out best parameters

In [11]:
# Import dataset
x, y = data_import(datapathway)
# Pre-processing
model_type, H_X_train, H_Y_train = H_pre_processing('ANN2', x, y)
# Run Bayesian Optimization: find maximum value of unknown function by a few iteration
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=5)
nn_bo.maximize(init_points=no_iter_before_conclude, n_iter=no_iter_conclude)
params_nn_ = nn_bo.max['params']
print(nn_bo)

|   iter    |  target   | Batch_... | N_hidd... |  epochs   |  l_rate   |
-------------------------------------------------------------------------
| 1         | -0.00949  | 14.99     | 1.743e+03 | 246.4     | 0.009194  |
| 2         | -0.004029 | 31.77     | 1.227e+03 | 777.6     | 0.005232  |
| 3         | -0.00701  | 19.7      | 383.6     | 126.7     | 0.007411  |
| 4         | -0.008331 | 28.8      | 325.0     | 885.9     | 0.002813  |
| 5         | -0.01269  | 27.1      | 599.2     | 647.3     | 0.00584   |
| 6         | -0.003256 | 31.24     | 1.225e+03 | 782.8     | 0.001124  |
| 7         | -0.005271 | 26.76     | 1.212e+03 | 818.4     | 0.007304  |
| 8         | -0.004179 | 57.78     | 1.207e+03 | 784.4     | 0.00112   |


### Step 11: Print out all related information from best hyperparameter combination

In [12]:
print(nn_bo.max)

{'target': -0.0032559549435973167, 'params': {'Batch_size': 31.24466415103516, 'N_hidden_nodes': 1225.0286866845124, 'epochs': 782.7979890818565, 'l_rate': 0.0011242803540236787}}
